## Question 6

### Importing relevant libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta, date
import datetime as datetime
from dateutil.relativedelta import relativedelta
from dateutil import rrule
import undetected_chromedriver as uc
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import warnings
import math
%matplotlib inline
warnings.filterwarnings('ignore')

C:\Users\MANAS\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\MANAS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\MANAS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


C:\Users\MANAS\anaconda3\lib\site-packages\pyfolio\pos.py:26: UserWarning:

Module "zipline.assets" not found; multipliers will not be applied to position notionals.



### Fetching Data

In [27]:
driver = uc.Chrome()
driver.get(f'https://www.nseindia.com/option-chain')
time.sleep(3)

In [28]:
ltp =float(driver.find_elements(By.XPATH, '//*[@id="equity_underlyingVal"]')[0].text.split(' ')[-1].replace(',',''))
print(f"Last Traded Price of Nifty50 : {ltp}")

Last Traded Price of Nifty50 : 16278.5


In [ ]:
download_btn = WebDriverWait(driver, 10).until(lambda x: x.find_element_by_xpath(f'//*[@id="downloadOCTable"]'))
download_btn.click()

In [25]:
def round_nearest(x,num=50): return int(math.ceil(float(x)/num)*num)

In [91]:
df = pd.DataFrame()
temp = pd.DataFrame()
for f in os.listdir(r'C:/Users/Manas/Edelweiss/Q6'):
    if f.endswith('.csv'):
        temp = pd.read_csv(f'C:/Users/Manas/Edelweiss/Q6/{f}',skiprows=1)  
        temp = temp.iloc[:,1:-1]
        df = pd.concat([df,temp],ignore_index=True)

In [92]:
df

,OI,CHNG IN OI,VOLUME,IV,LTP,CHNG,BID QTY,BID PRICE,ASK PRICE,ASK QTY,...,BID QTY.1,BID PRICE.1,ASK PRICE.1,ASK QTY.1,CHNG.1,LTP.1,IV.1,VOLUME.1,CHNG IN OI.1,OI.1
0,-,-,-,-,-,-,"1,250","2,458.05","2,518.55",250,...,"3,500",0.60,0.70,"12,300",-0.05,0.70,65.62,"8,777","1,631","9,469"
1,-,-,-,-,-,-,"1,250","2,375.85","2,466.75",450,...,50,0.60,0.80,600,0.10,0.80,65.16,"4,844",324,963
2,-,-,-,-,-,-,"1,250","2,337.20","2,421.25",500,...,600,0.60,0.75,600,-,0.75,63.44,"1,492",248,741
3,-,-,-,-,-,-,"2,250","2,276.65","2,396.40","1,000",...,"3,700",0.50,0.95,"1,500",-0.20,0.50,59.77,"4,992",51,292
4,-,-,3,-,"2,240.15",767.65,"1,250","2,242.90","2,344.45",200,...,"7,700",0.65,0.70,"36,600",-0.20,0.65,59.98,"32,030","-8,371","41,680"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,179,104,913,38.57,0.50,-0.20,"3,800",0.45,0.65,"2,300",...,"1,000","1,608.60","1,715.45","1,250",-,-,-,-,-,-
84,"40,024","12,095","86,872",39.57,0.50,-0.25,"56,950",0.50,0.55,"1,18,400",...,200,"1,661.95","1,706.65",150,-207.00,"1,715.00",56.19,5,5,109
85,180,58,"5,278",40.55,0.50,-0.25,"2,050",0.50,0.70,"1,000",...,"1,000","1,706.50","1,817.10","1,250",-,"1,991.00",-,-,-,2
86,"3,135",-285,"2,557",41.92,0.55,-0.10,"5,600",0.50,0.55,"4,950",...,"1,000","1,755.20","1,856.30","1,250",-,"2,135.00",-,-,-,1


In [93]:
atm_strike = round_nearest(ltp)

In [94]:
atm_strike

16300

### Formatting Data

In [95]:
for i in range(df.shape[0]):
    df.loc[i,'STRIKE PRICE'] = df.loc[i,'STRIKE PRICE'].replace(',','')
    df.loc[i,'OI'] = df.loc[i,'OI'].replace(',','')
    df.loc[i,'OI'] = df.loc[i,'OI'].replace('-','0')

    df.loc[i,'OI.1'] = df.loc[i,'OI.1'].replace(',','')
    df.loc[i,'OI.1'] = df.loc[i,'OI.1'].replace('-','0')

In [96]:
df['STRIKE PRICE'] = df['STRIKE PRICE'].astype(float)
df['OI'] = df['OI'].astype(int)
df['OI.1'] = df['OI.1'].astype(int)

### Computing required information

In [97]:
data_CE = df[(df['STRIKE PRICE']>=ltp)]
data_CE = data_CE[data_CE['STRIKE PRICE']<=(1.025*ltp)]

In [99]:
highest_OI_CE = data_CE[data_CE.OI == max(data_CE['OI'])]['STRIKE PRICE']
highest_OI_CE

56    16600.0
Name: STRIKE PRICE, dtype: float64

In [100]:
data_PE = df[(df['STRIKE PRICE']<=ltp)]
data_PE = data_PE[data_PE['STRIKE PRICE']>=(0.975*ltp)]

In [101]:
data_PE

,OI,CHNG IN OI,VOLUME,IV,LTP,CHNG,BID QTY,BID PRICE,ASK PRICE,ASK QTY,...,BID QTY.1,BID PRICE.1,ASK PRICE.1,ASK QTY.1,CHNG.1,LTP.1,IV.1,VOLUME.1,CHNG IN OI.1,OI.1
42,9299,"-9,418","70,922",20.83,405.70,187.05,50,406.40,409.00,50,...,"1,900",10.05,10.20,"1,800",-52.55,10.05,19.09,"9,40,023","8,634",90468
43,6194,"-8,853","44,934",19.85,358.60,174.25,250,358.35,362.00,50,...,50,13.10,13.50,300,-64.35,13.80,18.71,"4,86,868","12,694",41417
44,34427,"-35,971","3,81,384",19.44,314.20,161.45,900,314.25,317.55,150,...,50,17.75,18.00,"7,450",-78.80,17.90,18.04,"13,29,727","39,435",127519
45,8286,"-13,838","1,61,042",18.68,270.00,145.45,"4,150",270.00,273.25,400,...,300,22.90,23.80,"1,550",-93.35,23.80,17.50,"5,48,032","26,895",42144
46,24803,"-33,628","7,18,606",18.04,228.00,128.00,250,228.05,230.00,"1,600",...,50,31.00,32.05,550,-109.65,32.00,17.08,"13,60,076","83,992",102265
47,10293,"-12,056","5,71,266",17.56,189.00,110.65,200,187.70,192.00,200,...,"3,000",42.00,43.45,50,-128.30,42.00,16.52,"8,22,380","39,916",44583
48,69929,"1,638","25,43,584",16.93,151.95,91.80,250,152.25,155.90,150,...,"2,450",55.55,57.00,950,-145.75,56.00,16.17,"20,79,137","1,08,159",116635
49,23237,"2,394","11,63,054",16.61,120.00,75.65,50,120.40,122.95,50,...,"8,300",73.20,75.00,"5,200",-160.05,75.00,16.06,"5,65,841","29,739",30606


In [102]:
data_PE['OI.1'] = data_PE['OI.1'].astype(int)

In [103]:
highest_OI_PE = data_PE[data_PE['OI.1'] == max(data_PE['OI.1'])]['STRIKE PRICE']
highest_OI_PE

44    16000.0
Name: STRIKE PRICE, dtype: float64